In [1]:
import torch
print(torch.__version__)

1.7.0+cu101


In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset, DataLoader, random_split
from torch import Tensor
from torch.nn import *
from torch.optim import SGD
from torch.nn.init import xavier_uniform_, kaiming_uniform_

In [5]:
#Define the Dataset
class CSVDataset(Dataset):
  def __init__(self, path):
    df = pd.read_csv(path, header=None)
    #Get the X and Y array 
    self.X = df.values[:,:-1]
    self.y = df.values[:,-1]
    #Ensure that the x and y values are float32
    self.X = self.X.astype('float32')
    
    self.y = LabelEncoder().fit_transform(self.y)
    self.y = self.y.astype('float32')
    self.y = self.y.reshape((len(self.y), 1))

  
  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    return [self.X[idx], self.y[idx]]

  def get_splits(self, test_pc=0.33):
    test_size = round(len(self.X) * test_pc)
    train_size = len(self.X) - test_size
    # random_split takes as input the dataset (self) and the length of splits to produce (As specified by train/test_size)
    # Will return splits for the train and test data. Split into 2, with the given sizes
    return random_split(self, [train_size, test_size])

In [21]:
# torch.nn.Module is the base class for all neural network modules. 
# Your models should subclass this class
class MLP(Module):
  def __init__(self, n_inputs):
    super(MLP, self).__init__()

    # Input to first hidden layer
    # Parameters are the size of the input and outputs to this layer
    self.hidden1 = Linear(n_inputs, 10)
    # The weights of the given layer are initialized after the layer is defined
    kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
    self.act1 = ReLU()

    # Second hidden layer
    self.hidden2 = Linear(10, 8)
    kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
    self.act2 = ReLU()

    # Third hidden layer
    self.hidden3 = Linear(8,1)
    xavier_uniform_(self.hidden3.weight)
    # Use sigmoid as this is binary classification
    self.act3 = Sigmoid()

    # Forward Propogation
  
  def forward(self, X):
    # Kinda resembles the Tensorflow Functional Syntax. In fact, its the same....
    X = self.hidden1(X)
    X = self.act1(X)
    X = self.hidden2(X)
    X = self.act2(X)
    X = self.hidden3(X)
    X = self.act3(X)
    return X

In [7]:
# Pytorch's DataLoader is an interable over a Dataset
def prepare_data(path):
  dataset = CSVDataset(path)
  train, test = dataset.get_splits()
  train_dl = DataLoader(train, batch_size=32, shuffle=True)
  test_dl = DataLoader(test, batch_size=1024, shuffle=False)
  return train_dl, test_dl

In [25]:
def train_model(train_dl, model, epochs):
  # Define optimizers and loss
  criterion = BCELoss()
  # Must pass in the parameters you want to optimize and the additional parameters
  optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)

  step = 0

  # Enumerate number of epochs
  for epoch in range(epochs):
    for i, (inputs, targets) in enumerate(train_dl):
      # Clear the Gradients (I.e set to zero)
      optimizer.zero_grad()

      # Compute the model output
      yhat = model(inputs)

      # Calculate loss
      loss = criterion(yhat, targets)

      # Perform backpropagation to get the gradients
      # Compute the gradient of the loss with respect to the parameters. This is done for all parameters
      loss.backward()

      # Update model weights based on the gradients obtained in the previous step
      optimizer.step()

    print(f'Step {step}/{epochs}...')
    step += 1

In [30]:
def evaluate_model(test_dl, model):
  preds, actuals = [], []
  for i, (inputs, targets) in enumerate(test_dl):
    yhat = model(inputs)
    # Retrieve the numpy array from yhat (Which is currently a pytorch tensor object)
    yhat = yhat.detach().numpy()

    # Convert the actual (Which is currently a tensor) into numpy and reshape to be same as the yhat shape
    actual = targets.numpy()
    actual = actual.reshape((len(actual), 1))
    # All yhats are currenly a decimal between 0 and 1. Hence, you round to get a whole number
    yhat = yhat.round()

    preds.append(yhat)
    actuals.append(actual)
  # Vstack will vertically combine all the numpy arrays. Works like how extends work for python list. 
  # All but the first dimension of the arrays have to be same
  preds, actuals = np.vstack(preds), np.vstack(actuals)
  # Calculate the Accuracy
  acc = accuracy_score(actuals, preds)
  return acc

In [10]:
def predict(data, model):
  # Convert data to tensor object (To pass in to pytorch)
  data = Tensor([data])
  yhat = model(data)
  #Retrieve np array
  yhat.detach().numpy()
  return yhat

In [12]:
# Prepare the Data
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/ionosphere.csv'
train_dl, test_dl = prepare_data(path)

In [19]:
# Inputs will be a list of inputs [[2,4,5], [3,5,6]]
# targets will be a list of labels [[0], [1]]
for i, (inputs, targets) in enumerate(test_dl):
  print(inputs)
  # We realize that there are 34 features per data
  print(inputs.shape)

tensor([[ 1.0000,  0.0000,  0.5011,  ...,  0.1056,  0.6067, -0.0809],
        [ 1.0000,  0.0000,  0.9818,  ...,  0.1646,  0.5884,  0.1708],
        [ 1.0000,  0.0000,  0.7152,  ..., -0.1068,  0.3042, -0.0518],
        ...,
        [ 1.0000,  0.0000,  0.8705,  ..., -0.0594,  0.5146,  0.1664],
        [ 0.0000,  0.0000, -1.0000,  ...,  1.0000,  1.0000, -1.0000],
        [ 1.0000,  0.0000,  0.8063,  ...,  0.2390, -0.2385,  0.3115]])
torch.Size([116, 34])


In [26]:
# Initialize your model with 34 nodes at the first layer (As there are 34 features)
model = MLP(34)
train_model(train_dl, model, 100)

Step 0/100...
Step 1/100...
Step 2/100...
Step 3/100...
Step 4/100...
Step 5/100...
Step 6/100...
Step 7/100...
Step 8/100...
Step 9/100...
Step 10/100...
Step 11/100...
Step 12/100...
Step 13/100...
Step 14/100...
Step 15/100...
Step 16/100...
Step 17/100...
Step 18/100...
Step 19/100...
Step 20/100...
Step 21/100...
Step 22/100...
Step 23/100...
Step 24/100...
Step 25/100...
Step 26/100...
Step 27/100...
Step 28/100...
Step 29/100...
Step 30/100...
Step 31/100...
Step 32/100...
Step 33/100...
Step 34/100...
Step 35/100...
Step 36/100...
Step 37/100...
Step 38/100...
Step 39/100...
Step 40/100...
Step 41/100...
Step 42/100...
Step 43/100...
Step 44/100...
Step 45/100...
Step 46/100...
Step 47/100...
Step 48/100...
Step 49/100...
Step 50/100...
Step 51/100...
Step 52/100...
Step 53/100...
Step 54/100...
Step 55/100...
Step 56/100...
Step 57/100...
Step 58/100...
Step 59/100...
Step 60/100...
Step 61/100...
Step 62/100...
Step 63/100...
Step 64/100...
Step 65/100...
Step 66/100...
Step 

In [51]:
#Doing this to show how the variables look like

preds, actuals = [], []
for i, (inputs, targets) in enumerate(test_dl):
    yhat = model(inputs)
    # Retrieve the numpy array from yhat (Which is currently a pytorch tensor object)
    yhat = yhat.detach().numpy()

    # Convert the actual (Which is currently a tensor) into numpy and reshape to be same as the yhat shape
    actual = targets.numpy()
    actual = actual.reshape((len(actual), 1))
    # All yhats are currenly a decimal between 0 and 1. Hence, you round to get a whole number
    yhat = yhat.round()

    preds.append(yhat)
    actuals.append(actual)
preds, actuals = np.vstack(preds), np.vstack(actuals)
print(preds)

[[1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]]


In [31]:
acc = evaluate_model(test_dl, model)

In [32]:
print(f'Accuracy is {acc}')

Accuracy is 0.9396551724137931


In [34]:
example = [1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1,0.03760,0.85243,-0.17755,0.59755,-0.44945,0.60536,-0.38223,0.84356,-0.38542,0.58212,-0.32192,0.56971,-0.29674,0.36946,-0.47357,0.56811,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300]
yhat = predict(example, model)
# Must round (To get a number 0 or 1), since yhat will be a decimal between 0 to 1. 
print(f'It predicted: {yhat.round()}')

It predicted: tensor([[1.]], grad_fn=<RoundBackward>)
